In [1]:
import pandas as pd
import ipywidgets as widgets
from collections import Counter

In [2]:
predictions = pd.read_csv('CSV/predictions/predictions.csv')
predictions.sort_values(by=['predicted_point_range', 'prob_3', 'prob_2', 'prob_1', 'prob_0'], ascending=False, inplace=True)

In [3]:
your_squad = []

def add_player(b):
    player_name = players.value
    if player_name in tuple(predictions['name']):
        your_squad.append(player_name)
    else:
        raise ValueError('player name not valid')
    
def remove_player(b):
    player_name = squad.value
    if player_name in tuple(predictions['name']):
        your_squad.remove(player_name)
    else:
        raise ValueError('Player name not valid')
    
def restart_list(b):
    your_squad.clear()
    
def view_full_squad(b):
    player_rows = [predictions[predictions['name'] == name] for name in your_squad]
    if not player_rows:
        raise ValueError('Squad list is empty.')
    squad = pd.concat(player_rows)
    squad['position'] = pd.Categorical(squad['position'], categories=['GK', 'DEF', 'MID', 'FWD'])
    squad.sort_values(by=['position', 'plays_for', 'name'], inplace=True)
    teams_in_squad = Counter(tuple(squad['plays_for']))
    if max(teams_in_squad.values()) > 3:
        raise ValueError('Invalid team distribution in squad list. Must have 3 or less players from the same team.')
    display(squad)
    
def make_transfer(b):
    budget = remaining_budget.value
    player_rows = [predictions[predictions['name'] == name] for name in your_squad]
    if not player_rows:
        raise ValueError('Squad list is empty.')
    squad = pd.concat(player_rows).sort_values(by=['predicted_point_range', 'prob_3', 'prob_2', 'prob_1', 'prob_0'])
    teams_in_squad = Counter(tuple(squad['plays_for']))
    if max(teams_in_squad.values()) > 3:
        raise ValueError('Invalid team distribution in squad list. Must have 3 or less players from the same team.')
    statement = True
    i = 0
    while statement:
        best_replacement = predictions.iloc[i]
        if best_replacement['name'] in tuple(squad['name']):
            i += 1
            continue
        budget_needed = best_replacement['value'] - budget
        players_to_replace = squad[(squad['value'] >= budget_needed) & (squad['position'] == best_replacement['position'])]
        if not tuple(players_to_replace.index):
            i += 1
            continue
        for j, player in players_to_replace.iterrows():
            teams_in_squad[player['plays_for']] = teams_in_squad[player['plays_for']] - 1
            teams_in_squad[best_replacement['plays_for']] = teams_in_squad[best_replacement['plays_for']] + 1
            if max(teams_in_squad.values()) > 3:
                continue
            elif (best_replacement['predicted_point_range'] >= player['predicted_point_range']) & (best_replacement['prob_{}'.format(int(best_replacement['predicted_point_range']))] > player['prob_{}'.format(int(best_replacement['predicted_point_range']))]):
                print('Replace:', player['name'])
                print('With:', best_replacement['name'])
                statement = False
                break
            else:
                continue
        i += 1
        continue

In [4]:
players = widgets.Combobox(
    placeholder='Choose Player',
    options=tuple(predictions['name'].sort_values()),
    description='Name:',
    ensure_option=True,
    disabled=False)

add_button = widgets.Button(
    description='Click to add player to your squad.',
    disabled=False,
    button_style='success',
    layout=widgets.Layout(width='50%'))

add_button.on_click(add_player)

display(players), display(add_button)
print()

Combobox(value='', description='Name:', ensure_option=True, options=('Aaron Connolly', 'Aaron Cresswell', 'Aar…

Button(button_style='success', description='Click to add player to your squad.', layout=Layout(width='50%'), s…

In [5]:
# if you would like to remove a player from your squad, you will first need to re-run this cell before and after
# to refresh your squad list in the text list.
squad = widgets.Combobox(
    placeholder='Choose Player',
    options=tuple(sorted(your_squad)),
    description='Name:',
    ensure_option=True,
    disabled=False)

remove_button = widgets.Button(
    description='Click to remove player from your squad.',
    disabled=False,
    button_style='warning',
    layout=widgets.Layout(width='50%'))

remove_button.on_click(remove_player)

display(squad), display(remove_button)
print()

Combobox(value='', description='Name:', ensure_option=True, placeholder='Choose Player')

Button(button_style='warning', description='Click to remove player from your squad.', layout=Layout(width='50%…

In [6]:
restart_button = widgets.Button(
    description='Click to reset squad list.',
    disabled=False,
    button_style='danger',
    layout=widgets.Layout(width='50%'))

restart_button.on_click(restart_list)

display(restart_button)
print()

Button(button_style='danger', description='Click to reset squad list.', layout=Layout(width='50%'), style=Butt…

In [12]:
view_full_squad_button = widgets.Button(
    description='Click to view your squad.',
    disabled=False,
    button_style='info',
    layout=widgets.Layout(width='50%'))

view_full_squad_button.on_click(view_full_squad)

display(view_full_squad_button)
print()

Button(button_style='info', description='Click to view your squad.', layout=Layout(width='50%'), style=ButtonS…

In [11]:
style = {'description_width': 'initial'}

remaining_budget = widgets.IntText(
    value=None,
    description='Remaining Budget:',
    disabled=False,
    style=style)

make_transfer_button = widgets.Button(
    description='Click to find best transfer.',
    disabled=False,
    button_style='info',
    layout=widgets.Layout(width='50%'))

make_transfer_button.on_click(make_transfer)

display(remaining_budget), display(make_transfer_button)
print()

IntText(value=0, description='Remaining Budget:', style=DescriptionStyle(description_width='initial'))

Button(button_style='info', description='Click to find best transfer.', layout=Layout(width='50%'), style=Butt…